In [386]:
import pandas as pd

df = pd.read_csv('../data/punisher_domain.csv')

df

,average,date,highest,lowest,order_count,volume
0,255300.00,2022-07-01,256550,253100,115,195
1,275000.00,2022-07-02,335000,245900,101,138
2,332450.00,2022-07-03,365000,243100,103,158
3,250150.00,2022-07-04,367350,90030,90,112
4,249161.92,2022-07-05,325000,230300,73,73
...,...,...,...,...,...,...
407,393000.00,2023-08-12,509450,361200,92,122
408,485500.00,2023-08-13,489000,381700,86,139
409,381700.00,2023-08-14,473600,380200,81,122
410,469900.00,2023-08-15,469900,452500,102,212


In [387]:
# df = df[['date', 'average']]

df

,average,date,highest,lowest,order_count,volume
0,255300.00,2022-07-01,256550,253100,115,195
1,275000.00,2022-07-02,335000,245900,101,138
2,332450.00,2022-07-03,365000,243100,103,158
3,250150.00,2022-07-04,367350,90030,90,112
4,249161.92,2022-07-05,325000,230300,73,73
...,...,...,...,...,...,...
407,393000.00,2023-08-12,509450,361200,92,122
408,485500.00,2023-08-13,489000,381700,86,139
409,381700.00,2023-08-14,473600,380200,81,122
410,469900.00,2023-08-15,469900,452500,102,212


In [388]:
import datetime

def str_to_datetime(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return datetime.datetime(year=year, month=month, day=day)

datetime_object = str_to_datetime('1986-03-19')
datetime_object

datetime.datetime(1986, 3, 19, 0, 0)

In [389]:
df['date'] = df['date'].apply(str_to_datetime)
df['date']

0     2022-07-01
1     2022-07-02
2     2022-07-03
3     2022-07-04
4     2022-07-05
         ...    
407   2023-08-12
408   2023-08-13
409   2023-08-14
410   2023-08-15
411   2023-08-16
Name: date, Length: 412, dtype: datetime64[ns]

In [390]:
df.index = df.pop('date')
df

,average,highest,lowest,order_count,volume
date,,,,,
2022-07-01,255300.00,256550,253100,115,195
2022-07-02,275000.00,335000,245900,101,138
2022-07-03,332450.00,365000,243100,103,158
2022-07-04,250150.00,367350,90030,90,112
2022-07-05,249161.92,325000,230300,73,73
...,...,...,...,...,...
2023-08-12,393000.00,509450,361200,92,122
2023-08-13,485500.00,489000,381700,86,139
2023-08-14,381700.00,473600,380200,81,122


In [391]:
import matplotlib.pyplot as plt

# Comment in Plot to show
# plt.plot(df.index, df['average'])

In [392]:
import numpy as np

def df_to_windowed_df(dataframe, first_date_str, last_date_str, n=3):
  first_date = str_to_datetime(first_date_str)
  last_date  = str_to_datetime(last_date_str)

  target_date = first_date
  
  dates = []
  X, Y = [], []

  last_time = False
  while True:
    df_subset = dataframe.loc[:target_date].tail(n+1)
    
    if len(df_subset) != n+1:
      print(f'Error: Window of size {n} is too large for date {target_date}')
      return

    values = df_subset['average'].to_numpy()
    x, y = values[:-1], values[-1]

    dates.append(target_date)
    X.append(x)
    Y.append(y)

    next_week = dataframe.loc[target_date:target_date+datetime.timedelta(days=7)]
    next_datetime_str = str(next_week.head(2).tail(1).index.values[0])
    next_date_str = next_datetime_str.split('T')[0]
    year_month_day = next_date_str.split('-')
    year, month, day = year_month_day
    next_date = datetime.datetime(day=int(day), month=int(month), year=int(year))
    
    if last_time:
      break
    
    target_date = next_date

    if target_date == last_date:
      last_time = True
    
  ret_df = pd.DataFrame({})
  ret_df['Target Date'] = dates
  
  X = np.array(X)
  for i in range(0, n):
    X[:, i]
    ret_df[f'Target-{n-i}'] = X[:, i]
  
  ret_df['Target'] = Y

  return ret_df

# Start day second time around: '2021-03-25'
windowed_df = df_to_windowed_df(df, 
                                '2022-07-04', 
                                '2023-08-16', 
                                n=3)
windowed_df.rename(columns = {"Target Date": "date"}, inplace=True)
windowed_df.index = windowed_df.pop('date')
windowed_df

,Target-3,Target-2,Target-1,Target
date,,,,
2022-07-04,255300.00,275000.00,332450.00,250150.00
2022-07-05,275000.00,332450.00,250150.00,249161.92
2022-07-06,332450.00,250150.00,249161.92,240898.94
2022-07-07,250150.00,249161.92,240898.94,245548.11
2022-07-08,249161.92,240898.94,245548.11,267626.30
...,...,...,...,...
2023-08-12,367163.86,351098.07,374900.00,393000.00
2023-08-13,351098.07,374900.00,393000.00,485500.00
2023-08-14,374900.00,393000.00,485500.00,381700.00


In [393]:
#Concat our windowed df and our initial df
df

result = pd.concat([df, windowed_df], axis=1)
columns_to_drop = ['Target', 'Target-2', 'Target-3']
result = result.drop(columns=columns_to_drop)
result = result.dropna()
result = result.rename(columns={'Target-1': 'target'})
result

,average,highest,lowest,order_count,volume,target
date,,,,,,
2022-07-04,250150.00,367350,90030,90,112,332450.00
2022-07-05,249161.92,325000,230300,73,73,250150.00
2022-07-06,240898.94,300000,230300,75,94,249161.92
2022-07-07,245548.11,308000,230200,68,74,240898.94
2022-07-08,267626.30,339800,230200,80,100,245548.11
...,...,...,...,...,...,...
2023-08-12,393000.00,509450,361200,92,122,374900.00
2023-08-13,485500.00,489000,381700,86,139,393000.00
2023-08-14,381700.00,473600,380200,81,122,485500.00


In [394]:
X = result.drop(columns=['target'])
y = result['target']

In [395]:
from sklearn.preprocessing import MinMaxScaler
# Standardize the features
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [396]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(X_scaled):
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_scaled[train_index], y_scaled[val_index]


In [397]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

# Build the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error', "mean_absolute_percentage_error"])

class LossHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

loss_history = LossHistory()

# Train the model
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
model.fit(X_train_reshaped, y_train, epochs=8, batch_size=32, validation_data=(X_val_reshaped, y_val), callbacks=[loss_history])


# Evaluate the model on the validation set
X_val_reshaped = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
loss = model.evaluate(X_val_reshaped, y_val)
print("Validation loss:", loss)


Epoch 1/8
11/11 [==============================] - 1s 22ms/step - loss: 0.1588 - mean_absolute_error: 0.3784 - mean_absolute_percentage_error: 40412.7422 - val_loss: 0.1475 - val_mean_absolute_error: 0.3793 - val_mean_absolute_percentage_error: 90.6448
Epoch 2/8
11/11 [==============================] - 0s 4ms/step - loss: 0.1294 - mean_absolute_error: 0.3385 - mean_absolute_percentage_error: 130167.5625 - val_loss: 0.1125 - val_mean_absolute_error: 0.3299 - val_mean_absolute_percentage_error: 78.6534
Epoch 3/8
11/11 [==============================] - 0s 4ms/step - loss: 0.0899 - mean_absolute_error: 0.2748 - mean_absolute_percentage_error: 427704.8750 - val_loss: 0.0593 - val_mean_absolute_error: 0.2359 - val_mean_absolute_percentage_error: 55.8042
Epoch 4/8
11/11 [==============================] - 0s 4ms/step - loss: 0.0326 - mean_absolute_error: 0.1441 - mean_absolute_percentage_error: 1039371.8125 - val_loss: 0.0044 - val_mean_absolute_error: 0.0424 - val_mean_absolute_percentage_er

In [398]:
# Comment in Plot to show
# plt.plot(loss_history.losses, label='Training Loss')
# plt.plot(loss_history.val_losses, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Loss Over Time')
# plt.legend()
# plt.show()

In [399]:
    predicted_scaled_y = model.predict(X_val)

    # Inverse transform the scaled predictions to get them in original units
    predicted_original_y = scaler_y.inverse_transform(predicted_scaled_y)
    original_y = scaler_y.inverse_transform(y_val)

    # Plot the actual y values and the predicted y values
    # Comment in Plot to show
    # plt.figure(figsize=(12, 6))
    # plt.plot(result.index[val_index], original_y, label='Actual Values')
    # plt.plot(df.index[val_index], predicted_original_y, label='Predicted Values', linestyle='dashed')
    # plt.xlabel('Date')
    # plt.ylabel('Target Value')
    # plt.title('Actual vs. Predicted Values')
    # plt.legend()
    # plt.show()

3/3 [==============================] - 0s 1ms/step
